<a href="https://colab.research.google.com/github/Nathan-Roll1/PSST/blob/main/Tutorials/PSST_Q_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Necessary Packages

In [1]:
!python -m pip install optimum[onnxruntime]@git+https://github.com/Nathan-Roll1/optimum-CPU-PSST
# Custom fork to overcome CuPy requirement on CPU

!pip install transformers==4.35.2

  Cloning https://github.com/Nathan-Roll1/optimum-CPU-PSST to /tmp/pip-install-ydbz2w51/optimum_60f9520ae7fd4286b9a8acf263ce7d4b
  Running command git clone --filter=blob:none --quiet https://github.com/Nathan-Roll1/optimum-CPU-PSST /tmp/pip-install-ydbz2w51/optimum_60f9520ae7fd4286b9a8acf263ce7d4b
  Resolved https://github.com/Nathan-Roll1/optimum-CPU-PSST to commit 64e9f3389eaf341669d3c00ba336789dbd4f13f9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/

Import modules

In [2]:
from transformers import pipeline, AutoTokenizer, AutoFeatureExtractor
from optimum.onnxruntime import (
    AutoQuantizationConfig,
    ORTModelForSpeechSeq2Seq,
    ORTQuantizer
)

cupy-cuda11x uninstalled successfully


Create inference Pipeline

In [3]:
model_dir = "NathanRoll/psst-medium-en-q"

model = ORTModelForSpeechSeq2Seq.from_pretrained(model_dir, provider="CPUExecutionProvider")
tokenizer = AutoTokenizer.from_pretrained(model_dir)
feature_extractor = AutoFeatureExtractor.from_pretrained(model_dir)
cls_pipeline_onnx = pipeline("automatic-speech-recognition", model=model, tokenizer=tokenizer, feature_extractor=feature_extractor)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

encoder_model_quantized.onnx:   0%|          | 0.00/313M [00:00<?, ?B/s]

decoder_model_quantized.onnx:   0%|          | 0.00/514M [00:00<?, ?B/s]

decoder_with_past_model_quantized.onnx:   0%|          | 0.00/463M [00:00<?, ?B/s]

...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


generation_config.json:   0%|          | 0.00/330 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generate transcription! (*!!!!! = IU Boundary*)

In [5]:
inference_file = "gettysburg.wav"
cls_pipeline_onnx(inference_file)

{'text': 'Four score and seven years ago!!!!! our fathers brought forth on this continent!!!!! a new nation!!!!! conceived in liberty!!!!! and dedicated to the proposition!!!!! that all men are created equal!!!!! Now we are engaged in a great civil war!!!!! testing whether that nation!!!!! or any nation so conceived and so dedicated!!!!! can long endure'}

Also works with librosa, useful to feed audio longer than 30s!

In [ ]:
import librosa
y, sr = librosa.load('gettysburg.wav')

In [ ]:
cls_pipeline_onnx(y)

{'text': 'Four score and seven years ago!!!!! our fathers brought forth on this continent!!!!! a new nation!!!!! conceived in liberty!!!!! and dedicated to the proposition!!!!! that all men are created equal!!!!! Now we are engaged in a great civil war!!!!! testing whether that nation!!!!! or any nation so conceived and so dedicated!!!!! can long endure'}